In [1]:
%matplotlib inline
import librosa
import librosa.display
import IPython.display as ipd
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report
import pywt
print(pywt.families(short=False))
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from collections import Counter
import scipy
from sklearn.metrics import roc_auc_score
pywt.wavelist('db')
import tensorflow as tf

['Haar', 'Daubechies', 'Symlets', 'Coiflets', 'Biorthogonal', 'Reverse biorthogonal', 'Discrete Meyer (FIR Approximation)', 'Gaussian', 'Mexican hat wavelet', 'Morlet wavelet', 'Complex Gaussian wavelets', 'Shannon wavelets', 'Frequency B-Spline wavelets', 'Complex Morlet wavelets']


### Feature extraction method

In [2]:
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy
 
def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]
 
def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values) > 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]
 
def get_features(list_values):
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

def get_audio_features(df, waveletname):
    list_features = []
    for audio_file in df['filepath']:
        signal=librosa.load(audio_file, sr=None)[0]
        list_coeff = pywt.wavedec(signal, waveletname)
        features = []
        for coeff in list_coeff:
            features += get_features(coeff)
        list_features.append(features)
    return pd.DataFrame(list_features)

### Import data method

In [3]:
PATH_TRAINING_DATASET = './Datasets/Training/'
PATH_ADDITIONNAL_DATASET = './Datasets/Additionnal/'

#Get list of files in train and test directory
def get_df_files(type_machinery):


    path_train_folder = PATH_TRAINING_DATASET + 'dev_' + type_machinery + '/' + type_machinery + '/train/'
    path_test_folder = PATH_TRAINING_DATASET + 'dev_' + type_machinery + '/' + type_machinery + '/test/'

    train_files = [f for f in os.listdir(path_train_folder)]
    test_files = [f for f in os.listdir(path_test_folder)]

    #Get list of dictionnary for creating DataFrame
    list_dict_file = []

    #Loop through filenames
    for filename in train_files:

        #Get filename as list of string
        splitted_filename = filename.split('_')

        #Append dictionnary to list
        list_dict_file.append({
            'filepath' : path_train_folder + filename,
            'filename' : filename,
            'section' : int(splitted_filename[1]),
            'domain_env' : splitted_filename[2],
            'dir' : splitted_filename[3],
            'sound_type' : splitted_filename[4],
            'id' : splitted_filename[5],
            'suffix' : '_'.join(splitted_filename[6:]).split('.wav')[0]
        })

    #Loop through filenames
    for filename in test_files:

        #Get filename as list of string
        splitted_filename = filename.split('_')

        #Append dictionnary to list
        list_dict_file.append({
            'filepath' : path_test_folder + filename,
            'filename' : filename,
            'section' : int(splitted_filename[1]),
            'domain_env' : splitted_filename[2],
            'dir' : splitted_filename[3],
            'sound_type' : splitted_filename[4],
            'id' : splitted_filename[5],
            'suffix' : '_'.join(splitted_filename[6:]).split('.wav')[0]
        })

    #Get file dataframe
    df_files = pd.DataFrame(list_dict_file)
    
    return df_files

# Valve

## Get df of features

In [4]:
#Get df files
df = get_df_files('valve')
df['target'] = df['sound_type'].replace(['normal', 'anomaly'], [0, 1])

#Encode suffix label
label_encoder = LabelEncoder()
df['suffix_as_int'] = label_encoder.fit_transform(df['suffix'])

#Take 150 normal rows section 0
normal_sect_0 = df[((df['section'] == 0 ) & (df['sound_type'] == 'normal' ))].sample(n=150, weights='suffix_as_int')

#Take 150 normal rows section 1
normal_sect_1 = df[((df['section'] == 1 ) & (df['sound_type'] == 'normal' ))].sample(n=150, weights='suffix_as_int')

#Take 150 normal rows section 2
normal_sect_2 = df[((df['section'] == 2 ) & (df['sound_type'] == 'normal' ))].sample(n=150, weights='suffix_as_int')

#Take the 300 anormal rows (100 for each section)
anormal = df[df['sound_type'] == 'anomaly']

#Concat together
df_selected = pd.concat([normal_sect_0, normal_sect_1, normal_sect_2, anormal])
df_selected.shape

(750, 10)

In [5]:
#Let's get train and test dataset
X_train,X_test,y_train,y_test = train_test_split(get_audio_features(df_selected, 'db8'), 
                                                 df_selected['target'],test_size=0.2) # change 'db8' for other type of wavelets

In [6]:
X_train.shape

(600, 168)

## Model initialization

In [11]:
#Initializaiton of the layers
inputs = Input(shape=X_train.shape[1], name='Inputs')
first_layer = Dense(64, activation='relu', kernel_initializer ='normal', name='First_layer')
second_layer = Dense(64, activation='relu', kernel_initializer ='normal', name='Second_layer')
third_layer = Dense(1, activation='sigmoid', kernel_initializer ='normal', name='Output_layer')

#Construct the layer's order
x = first_layer(inputs)
x = second_layer(x)
outputs = third_layer(x)

#Initialize the model
model = Model(inputs=inputs, outputs=outputs)
model.summary()

#Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Inputs (InputLayer)         [(None, 168)]             0         
                                                                 
 First_layer (Dense)         (None, 64)                10816     
                                                                 
 Second_layer (Dense)        (None, 64)                4160      
                                                                 
 Output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 15,041
Trainable params: 15,041
Non-trainable params: 0
_________________________________________________________________


## Model Training

In [12]:
model.fit(X_train, y_train, epochs=100, batch_size=30, validation_split=0.2)

Epoch 1/100
16/16 [==============================] - 0s 10ms/step - loss: 88.9247 - accuracy: 0.5167 - val_loss: 52.3990 - val_accuracy: 0.5583
Epoch 2/100
16/16 [==============================] - 0s 5ms/step - loss: 19.4193 - accuracy: 0.5271 - val_loss: 11.5906 - val_accuracy: 0.4417
Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 4.6283 - accuracy: 0.5271 - val_loss: 1.9068 - val_accuracy: 0.5500
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 1.7764 - accuracy: 0.5375 - val_loss: 3.4701 - val_accuracy: 0.5583
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 4.9097 - accuracy: 0.5521 - val_loss: 7.7456 - val_accuracy: 0.5583
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 6.9470 - accuracy: 0.5500 - val_loss: 6.7063 - val_accuracy: 0.5583
Epoch 7/100
16/16 [==============================] - 0s 5ms/step - loss: 3.3009 - accuracy: 0.5396 - val_loss: 4.1689 - val_accuracy: 0.4417
Epoch 8/

Epoch 59/100
16/16 [==============================] - 0s 5ms/step - loss: 1.2925 - accuracy: 0.5063 - val_loss: 2.1821 - val_accuracy: 0.4417
Epoch 60/100
16/16 [==============================] - 0s 5ms/step - loss: 1.4896 - accuracy: 0.5354 - val_loss: 0.9765 - val_accuracy: 0.5417
Epoch 61/100
16/16 [==============================] - 0s 4ms/step - loss: 1.2299 - accuracy: 0.5354 - val_loss: 1.7378 - val_accuracy: 0.4417
Epoch 62/100
16/16 [==============================] - 0s 5ms/step - loss: 1.5703 - accuracy: 0.5083 - val_loss: 1.6987 - val_accuracy: 0.5583
Epoch 63/100
16/16 [==============================] - 0s 4ms/step - loss: 1.2424 - accuracy: 0.5208 - val_loss: 2.3138 - val_accuracy: 0.5583
Epoch 64/100
16/16 [==============================] - 0s 5ms/step - loss: 1.4945 - accuracy: 0.5500 - val_loss: 0.9245 - val_accuracy: 0.5583
Epoch 65/100
16/16 [==============================] - 0s 5ms/step - loss: 0.8414 - accuracy: 0.5604 - val_loss: 1.3256 - val_accuracy: 0.5583
Epoch 

## Evaluation

In [13]:
test_pred = model.predict(X_test)
test_pred_class = test_pred.argmax(axis=1)

print(classification_report(y_test, test_pred_class))

              precision    recall  f1-score   support

           0       0.59      1.00      0.74        89
           1       0.00      0.00      0.00        61

    accuracy                           0.59       150
   macro avg       0.30      0.50      0.37       150
weighted avg       0.35      0.59      0.44       150



C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Quentin\anaconda3\envs\tensorflow_keras\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta